In [12]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2 # https://ipython.org/ipython-doc/3/config/extensions/autoreload.html

import numpy as np
import numpy.ma as ma
import h5py

import matplotlib.pyplot as plt
from keras.models import load_model


import sys
sys.path.insert(0, '../api_tutorials')
from bqsr_cnn import parse_args, read_to_bqsr_tensor, bqsr_get_metric_dict, bqsr_bed_file_to_dict, BQSR_LABELS
from bqsr_cnn import bqsr_get_train_valid_test_paths_all, bqsr_label_tensors_generator, OQ_TENSOR_NAME, BQSR_TENSOR_NAME
from bqsr_cnn import label_bases_model_from_args
from recal_utils import tensor_to_quality_array, KL_divergence

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
sys.argv = ['bqsr_cnn']

args = parse_args()
args.skip_connections = [0, -1, -2, -3, -4, -5]
args.conv_layers = [128, 128, 128, 128, 128, 128]

Arguments are Namespace(activation='relu', activity_weight=1.0, annotation_set='_', annotations=[], bam_file=None, base_quality_mode='phot', batch_size=32, bed_file=None, channels_last=False, chrom=None, conv_batch_normalize=False, conv_dropout=0.0, conv_layers=[128, 96, 64, 48], conv_widths=[5, 5, 5, 5], data_dir=None, downsample_perfect_reads=0.0, end_pos=0, epochs=25, fc_batch_normalize=False, fc_dropout=0.0, fc_initializer='glorot_normal', fc_layers=[32], fps=1, id='no_id', ignore_vcf=None, input_symbol_set='dna_annotations', input_symbols={'A': 0, 'C': 1, 'G': 2, 'T': 3, 'pair': 4, 'cycle': 5, 'mq': 6}, inspect_model=False, inspect_show_labels=False, iterations=5, jitter=0.0, kernel_initializer='glorot_normal', l1=0.0, l2=0.0, label_smoothing=0.0, label_weights=[0.5, 0.5], labels={'GOOD_BASE': 0, 'BAD_BASE': 1}, learning_rate=0.01, map_input_to_logspace=False, max_pools=[], maxfun=9, mode=None, model=None, out='recalibrated.bam', output_dir='./weights/', padding='same', patience=4

In [3]:
model = label_bases_model_from_args(args)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bqsr (InputLayer)               (None, 151, 7)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 151, 128)     4608        bqsr[0][0]                       
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 151, 128)     82048       conv1d_1[0][0]                   
__________________________________________________________________________________________________
add_1 (Add)                     (None, 151, 128)     0           conv1d_2[0][0]                   
                                                                 conv1d_1[0][0]                   
__________

In [25]:
args.data_dir ='/home/tsato/data/tensors/remove-all-error-reads-mini/'
# args.data_dir = '/home/tsato/data/tensors/remove-q2/'
train_paths, valid_paths, test_paths = bqsr_get_train_valid_test_paths_all(args.data_dir)
generate_train = bqsr_label_tensors_generator(args, train_paths)

In [27]:
next(generate_train)

({'oq': array([[[  9.99369043e-01,   2.10319115e-04,   2.10319115e-04, ...,
             1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
          [  2.10319115e-04,   9.99369043e-01,   2.10319115e-04, ...,
             1.00000000e+00,   6.62251656e-03,   1.00000000e+00],
          [  6.65087438e-05,   6.65087438e-05,   6.65087438e-05, ...,
             1.00000000e+00,   1.32450331e-02,   1.00000000e+00],
          ..., 
          [  2.10319115e-05,   9.99936904e-01,   2.10319115e-05, ...,
             1.00000000e+00,   9.80132450e-01,   1.00000000e+00],
          [  9.99936904e-01,   2.10319115e-05,   2.10319115e-05, ...,
             1.00000000e+00,   9.86754967e-01,   1.00000000e+00],
          [  2.10319115e-04,   2.10319115e-04,   2.10319115e-04, ...,
             1.00000000e+00,   9.93377483e-01,   1.00000000e+00]],
  
         [[  2.10319115e-04,   9.99369043e-01,   2.10319115e-04, ...,
             1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
          [  9.9936904

In [18]:
# tp = train_paths[0]
# tp = '/home/tsato/data/tensors/remove-all-error-reads/valid/NA12878_nist_na12878_minimal-HK35M.31_39752710.hd5'
# tp = '/home/tsato/data/tensors/remove-all-error-reads/train/NA12878_nist_na12878_minimal-HJYFJ.41_10087305.hd5'
tp = '/home/tsato/data/tensors/remove-all-error-reads-mini/train/NA12878_nist_na12878_minimal-HJYFJ.46_1000922.hd5'

with h5py.File(tp, 'r') as hf:
    tensor = np.array(hf.get(OQ_TENSOR_NAME))
    bqsr_tensor = np.array(hf.get(BQSR_TENSOR_NAME))

In [28]:
print(tensor.shape)
print(bqsr_tensor.shape)


(151, 7)
(151, 7)


Namespace(activation='relu', activity_weight=1.0, annotation_set='_', annotations=[], bam_file=None, base_quality_mode='phot', batch_size=32, bed_file=None, channels_last=False, chrom=None, conv_batch_normalize=False, conv_dropout=0.0, conv_layers=[128, 128, 128, 128, 128, 128], conv_widths=[5, 5, 5, 5], data_dir='/home/tsato/data/tensors/remove-all-error-reads-mini/', downsample_perfect_reads=0.0, end_pos=0, epochs=25, fc_batch_normalize=False, fc_dropout=0.0, fc_initializer='glorot_normal', fc_layers=[32], fps=1, id='no_id', ignore_vcf=None, input_symbol_set='dna_annotations', input_symbols={'A': 0, 'C': 1, 'G': 2, 'T': 3, 'pair': 4, 'cycle': 5, 'mq': 6}, inspect_model=False, inspect_show_labels=False, iterations=5, jitter=0.0, kernel_initializer='glorot_normal', l1=0.0, l2=0.0, label_smoothing=0.0, label_weights=[0.5, 0.5], labels={'GOOD_BASE': 0, 'BAD_BASE': 1}, learning_rate=0.01, map_input_to_logspace=False, max_pools=[], maxfun=9, mode=None, model=None, out='recalibrated.bam', o

In [ ]:
import pysam
import vcf
from bqsr_cnn import write_reads_in_region_to_tensors
from Bio import SeqIO

args.bam_file = '/home/tsato/bam/NA12878.bam'
args.ignore_vcf = '/home/tsato/auxiliary/nist_na12878_minimal.vcf.gz'
args.bed_file = '/home/tsato/auxiliary/union13callableMQonlymerged_addcert_nouncert_excludesimplerep_excludesegdups_excludedecoy_excludeRepSeqSTRs_noCNVs_v2.18_2mindatasets_5minYesNoRatio.bed'
args.reference_fasta = '/home/tsato/reference/Homo_sapiens_assembly19.fasta'

samfile = pysam.AlignmentFile(args.bam_file, "rb")
vcf_ram = vcf.Reader(open(args.ignore_vcf, 'rb'))

bed_dict = bqsr_bed_file_to_dict(args.bed_file)
record_dict = SeqIO.to_dict(SeqIO.parse(args.reference_fasta, "fasta"))


In [ ]:
write_reads_in_region_to_tensors(args, samfile, contigs[i], c, start, stop - margin, stats)